# Tube Learning Notebook

## DataFrame Construction

In [1]:
import pandas as pd
import ast
import numpy as np
import glob

In [2]:
import pandas as pd
import numpy as np
import pickle
import os

local_folder = 'rom_tracking_data/fxqv2jkf'

# Load the pickle file
with open(f"{local_folder}/dataset.pickle", "rb") as f:
    data = pickle.load(f)

# Create DataFrame from the pickle data
df = pd.DataFrame({
    'z': list(data['z']),
    'v': list(data['v']),
    'pz_x': list(data['pz_x']),
    'z_p1': list(data['z_p1']),
    'pz_x_p1': list(data['pz_x_p1']),
})

# Compute the vector differences for w_xy_t and its norm w_t
df['w_xy'] = [tuple(np.array(pz) - np.array(z)) for pz, z in zip(df['pz_x'], df['z'])]
df['w'] = [np.linalg.norm(np.array(pz) - np.array(z)) for pz, z in zip(df['pz_x'], df['z'])]

# Compute the vector differences for the next timestep w_xy_{t+1} and its norm w_{t+1}
df['w_xy_p1'] = [tuple(np.array(pz) - np.array(z)) for pz, z in zip(df['pz_x_p1'], df['z_p1'])]
df['w_t_p1'] = [np.linalg.norm(np.array(pz) - np.array(z)) for pz, z in zip(df['pz_x_p1'], df['z_p1'])]

# Cleanup: Drop the intermediate columns if they are no longer needed
df.drop(['pz_x', 'z_p1', 'pz_x_p1'], axis=1, inplace=True)

# Print to check the first few rows
print(df.head())

                                             z  \
0  [0.015644509840537696, 0.01509342525270101]   
1   [0.03128901968107539, 0.03018685050540202]   
2  [0.04693352952161309, 0.045280275758103034]   
3   [0.06257803936215078, 0.06037370101080404]   
4   [0.07822254920268848, 0.07546712626350505]   

                                          v  \
0  [0.7822254920268847, 0.7546712626350505]   
1  [0.7822254920268847, 0.7546712626350505]   
2  [0.7822254920268847, 0.7546712626350505]   
3  [0.7822254920268847, 0.7546712626350505]   
4  [0.7822254920268847, 0.7546712626350505]   

                                            w_xy         w  \
0   (-0.00878550348036447, -0.02359656427171346)  0.025179   
1  (-0.015280707798460302, -0.04643722057574027)  0.048887   
2  (-0.022484283517389264, -0.06901448954244008)  0.072585   
3  (-0.028610522045809383, -0.08874247581229673)  0.093240   
4   (-0.03388549635077312, -0.10423396509719474)  0.109604   

                                         w_

In [2]:
# Function to safely evaluate lists
def safe_eval(col):
    try:
        return ast.literal_eval(col)
    except ValueError:
        return col  # Return as is if it's not a string representation of a list

# Initialize an empty DataFrame
all_data = pd.DataFrame()

# Set the number of robots you want to include
num_robots = 5  # Set the number of robots to include
robot_indices = list(range(num_robots))  # Generates a list [0, 1, 2, ..., num_robots-1]

# Use glob to find all the files that match the pattern
file_list = glob.glob('data/trajectory_data_*.csv')

# Loop through the files sorted to maintain the order
for filename in sorted(file_list):
    temp_df = pd.read_csv(filename)
    # Filter the DataFrame to only include rows where the robot_index is in the list of desired indices
    temp_df = temp_df[temp_df['robot_index'].isin(robot_indices)]
    # Apply transformations right after reading
    temp_df['joint_positions'] = temp_df['joint_positions'].apply(safe_eval)
    temp_df['joint_velocities'] = temp_df['joint_velocities'].apply(safe_eval)
    all_data = pd.concat([all_data, temp_df], ignore_index=True)


In [3]:
# Now create the derived columns
all_data['x_t'] = all_data.apply(lambda row: row['joint_positions'] + row['joint_velocities'], axis=1)
all_data['u_t'] = all_data.apply(lambda row: [row['velocity_x'], row['velocity_y']], axis=1)
all_data['z_t'] = all_data.apply(lambda row: [row['traj_x'], row['traj_y']], axis=1)
all_data['v_t'] = all_data.apply(lambda row: [row['reduced_command_x'], row['reduced_command_y']], axis=1)

# Calculate the vector differences without norm for w_xy_t and shift it for w_xy_{t+1}
all_data['w_xy_t'] = all_data.apply(lambda row: [row['position_x'] - row['traj_x'], row['position_y'] - row['traj_y']], axis=1)
all_data['group'] = all_data['episode_number'].astype(str) + '_' + all_data['robot_index'].astype(str)

# Original calculation for w_t for reference
all_data['w_t'] = np.sqrt((all_data['position_x'] - all_data['traj_x'])**2 + (all_data['position_y'] - all_data['traj_y'])**2)

# Shift operations for next timestep values
all_data['x_{t+1}'] = all_data.groupby('group')['x_t'].shift(-1)
all_data['z_{t+1}'] = all_data.groupby('group')['z_t'].shift(-1)
all_data['w_{t+1}'] = all_data.groupby('group')['w_t'].shift(-1)
all_data['w_xy_{t+1}'] = all_data.groupby('group')['w_xy_t'].shift(-1)

# Function to drop the first and last 10 data points from each episode
def drop_edges(group):
    return group.iloc[10:-10]
all_data = all_data.groupby('group', group_keys=False).apply(drop_edges)

# Drop rows where x_{t+1}, z_{t+1}, w_{t+1}, and w_xy_{t+1} do not exist
all_data.dropna(subset=['x_{t+1}', 'z_{t+1}', 'w_{t+1}', 'w_xy_{t+1}'], inplace=True)

# Select and order the final columns
final_df = all_data[['group', 'x_t', 'u_t', 'z_t', 'v_t', 'w_t', 'w_xy_t', 'x_{t+1}', 'z_{t+1}', 'w_{t+1}', 'w_xy_{t+1}']]


ValueError: Cannot set a DataFrame with multiple columns to the single column x_t

We have $D=\{\omega_t, x_t, u_t, z_t, v_t, \omega_{t+1}, x_{t+1}, z_{t+1}\}$:

In [ ]:
# Print the final DataFrame
final_df.head()

Optional Saving:

In [ ]:
final_df.to_csv('processed_trajectory_data.csv', index=False)

## Network Construction

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import ast
from tqdm import tqdm
import wandb
import os

wandb.login(key="70954bb73c536b7f5b23ef315c7c19b511e8a406")

Load in the data:

In [ ]:
def safe_eval(col):
    try:
        return ast.literal_eval(col)
    except ValueError:
        return col  # Return as is if it's not a string representation of a list

def convert_to_tensor_input(row):
    flat_list = []
    for item in row:
        if isinstance(item, list):
            flat_list.extend(item)
        else:
            flat_list.append(item)
    return flat_list

def load_and_prepare_data(filename, tube_type):
    df = pd.read_csv(filename)
    list_columns = ['u_t', 'z_t', 'v_t'] + (['w_xy_t', 'w_xy_{t+1}'] if tube_type == 'rectangular' else [])
    feature_columns = ['u_t', 'z_t', 'v_t']
    
    for col in list_columns:
        df[col] = df[col].apply(safe_eval)  # Assumes safe_eval correctly parses strings into lists

    X = torch.tensor(df[feature_columns].apply(convert_to_tensor_input, axis=1).tolist(), dtype=torch.float32)
    
    if tube_type == 'rectangular':
        target_columns = ['w_xy_t', 'w_xy_{t+1}']
    else:
        target_columns = ['w_t', 'w_{t+1}']

    # Construct the target tensor manually from rows
    y_data = df[target_columns].apply(lambda row: [row[col] for col in target_columns], axis=1).tolist()
    y = torch.tensor(y_data, dtype=torch.float32)
    
    return X, y

def create_data_loaders(X, y, batch_size=64):
    # Create a TensorDataset
    dataset = TensorDataset(X, y[:, 1].unsqueeze(1))  # Assumes the target is at the second position

    # Split data into train and test sets
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

    # Create DataLoader objects
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    return train_loader, test_loader

Model specifications:

In [ ]:
class TubeWidthPredictor(nn.Module):
    def __init__(self, input_size, num_units, num_layers, output_dim=1):
        super(TubeWidthPredictor, self).__init__()
        self.layers = nn.ModuleList([nn.Linear(input_size, num_units), nn.ReLU()])
        for _ in range(num_layers - 1):
            self.layers.append(nn.Linear(num_units, num_units))
            self.layers.append(nn.ReLU())
        self.layers.append(nn.Linear(num_units, output_dim))  # Dynamically set output dimension

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

class AsymmetricLoss(nn.Module):
    def __init__(self, alpha=0.9, delta=1.0):
        super(AsymmetricLoss, self).__init__()
        self.alpha = alpha
        self.huber = nn.HuberLoss(delta=delta)

    def forward(self, y_pred, y_true, tube_type):
        if tube_type == 'sphere':
            residual = y_true - y_pred
            loss = torch.where(residual <= 0, self.alpha * residual, (1 - self.alpha) * residual.abs())
            return self.huber(loss, torch.zeros_like(loss))
        elif tube_type == 'rectangular':
            # Compute L2 norm of the residuals for each component
            y_true = y_true.squeeze(1) # because im a lazy idiot
            residual_x = y_true[:, 0] - y_pred[:, 0]
            residual_y = y_true[:, 1] - y_pred[:, 1]
            norm_residual = torch.sqrt(residual_x**2 + residual_y**2)
            loss = torch.where(norm_residual <= 0, self.alpha * norm_residual, (1 - self.alpha) * norm_residual.abs())
            return self.huber(loss, torch.zeros_like(loss))

def train_and_test(model, criterion, optimizer, train_loader, test_loader, tube_type, num_epochs=500):
    best_test_loss = float('inf')  # Initialize with a large value

    for epoch in tqdm(range(num_epochs), desc="Epochs", position=0, leave=True):
        model.train()
        train_loss = 0
        for data, targets in tqdm(train_loader, desc="Training Batches", leave=False):
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, targets, tube_type)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Log train loss to wandb
        wandb.log({'Train Loss': train_loss / len(train_loader), 'Epoch': epoch})

        # Evaluate the model
        test_loss, metrics = evaluate_model(model, test_loader, criterion, tube_type)
        
        # Log metrics to wandb
        wandb.log({'Test Loss': test_loss, **metrics, 'Epoch': epoch})
        
        # Checkpoint model if the test loss improved
        if test_loss < best_test_loss:
            best_test_loss = test_loss
            model_path = f"models/{tube_type}/model_epoch_{epoch}.pth"
            torch.save(model.state_dict(), model_path)
            wandb.save(model_path)  # Saves the checkpoint to wandb

def evaluate_model(model, test_loader, criterion, tube_type):
    model.eval()
    test_loss = 0
    differences = []
    total_predictions = 0  # Total number of predictions
    count_y_pred_gt_wt1 = 0  # Count of predictions greater than target

    with torch.no_grad():
        for data, targets in test_loader:
            outputs = model(data)
            test_loss += criterion(outputs, targets, tube_type).item()

            # Increase total predictions
            total_predictions += outputs.size(0)

            if tube_type == 'sphere':
                # Sphere-specific metric calculations
                greater_mask = outputs > targets
                count_y_pred_gt_wt1 += greater_mask.sum().item()
                # Calculate differences where predictions are less than targets
                less_mask = outputs < targets
                differences.extend((targets[less_mask] - outputs[less_mask]).abs().tolist())

            elif tube_type == 'rectangular':
                # Rectangular-specific metric calculations
                targets = targets.squeeze(1) # again... singular braincell human being...
                outputs_x_norm = outputs[:, 0]
                outputs_y_norm = outputs[:, 1]
                targets_x_norm = targets[:, 0]
                targets_y_norm = targets[:, 1]
                
                # Check if either x or y prediction is greater than the target
                greater_mask_x = outputs_x_norm > targets_x_norm
                greater_mask_y = outputs_y_norm > targets_y_norm
                count_y_pred_gt_wt1 += (greater_mask_x | greater_mask_y).sum().item()

                # Calculate differences for x and y separately where predictions are less than targets
                differences_x = (targets_x_norm - outputs_x_norm)[outputs_x_norm < targets_x_norm].abs().tolist()
                differences_y = (targets_y_norm - outputs_y_norm)[outputs_y_norm < targets_y_norm].abs().tolist()
                differences.extend(differences_x)
                differences.extend(differences_y)

    avg_diff = sum(differences) / len(differences) if differences else 0
    test_loss /= len(test_loader)
    proportion_y_pred_gt_wt1 = count_y_pred_gt_wt1 / total_predictions if total_predictions > 0 else 0
    
    metrics = {
        'Test Loss': test_loss,
        'Proportion y_pred > w_{t+1}': proportion_y_pred_gt_wt1,
        'Avg Abs Diff y_pred < w_{t+1}': avg_diff
    }
    return test_loss, metrics

In [ ]:
def main():
    wandb.init()
    config = wandb.config
    tube_type = config.tube_type
    filename = 'processed_trajectory_data.csv'
    
    X, y = load_and_prepare_data(filename, tube_type)
    train_loader, test_loader = create_data_loaders(X, y, batch_size=64)

    # Set input size based on tube type
    input_size = 6
    output_dim = 2 if tube_type == 'rectangular' else 1  # Set output dimension based on tube type

    model = TubeWidthPredictor(input_size=input_size, num_units=config.num_units, num_layers=config.num_layers, output_dim=output_dim)
    criterion = AsymmetricLoss(alpha=config.alpha, delta=1.0)
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
    
    train_and_test(model, criterion, optimizer, train_loader, test_loader, tube_type, num_epochs=config.num_epochs)

    wandb.finish()

In [ ]:
# Hyperparameter sweep and project setup
alpha_values = [0.8]
tube_types = ['rectangular', 'sphere']

for alpha in alpha_values:
    for tube_type in tube_types:
        sweep_config = {
            'method': 'grid',
            'metric': {'name': 'Test Loss', 'goal': 'minimize'},
            'parameters': {
                'alpha': {'value': alpha},
                'learning_rate': {'values': [0.001]},
                'num_units': {'values': [32]},
                'num_layers': {'values': [2]},
                'tube_type': {'value': tube_type},
                'num_epochs': {'value': 500}  # Example of defining epoch in config
            }
        }
        project_name = f"tube_width_experiment_alpha_{alpha}_{tube_type}"
        sweep_id = wandb.sweep(sweep_config, project=project_name)
        wandb.agent(sweep_id, main)